In [1]:
#import OpenAI key with helper function
from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

In [2]:
#A lot of modules use async and we want them to be compatible with Jupyter notebook
import nest_asyncio

nest_asyncio.apply()

In [15]:
import os
import glob

# Define the path to the directory containing the PDF files
folder_path = 'Ehlers-Danlos-1'

# Get the list of all PDF files in the directory
pdf_files = glob.glob(os.path.join(folder_path, '*.pdf'))
print(pdf_files)

# Extract just the filenames (optional)
pdf_filenames = [os.path.basename(pdf) for pdf in pdf_files]

# Print the list of PDF filenames
print(pdf_filenames)


['Ehlers-Danlos-1\\2024_EDS_1.pdf', 'Ehlers-Danlos-1\\2024_EDS_2.pdf', 'Ehlers-Danlos-1\\2024_EDS_3.pdf', 'Ehlers-Danlos-1\\2024_EDS_4.pdf', 'Ehlers-Danlos-1\\2024_EDS_5.pdf', 'Ehlers-Danlos-1\\2024_EDS_6.pdf', 'Ehlers-Danlos-1\\2024_EDS_7.pdf', 'Ehlers-Danlos-1\\Unknown_EDS_1.pdf', 'Ehlers-Danlos-1\\Unknown_EDS_2.pdf', 'Ehlers-Danlos-1\\Unknown_EDS_3.pdf', 'Ehlers-Danlos-1\\Unknown_EDS_4.pdf', 'Ehlers-Danlos-1\\Unknown_EDS_5.pdf']
['2024_EDS_1.pdf', '2024_EDS_2.pdf', '2024_EDS_3.pdf', '2024_EDS_4.pdf', '2024_EDS_5.pdf', '2024_EDS_6.pdf', '2024_EDS_7.pdf', 'Unknown_EDS_1.pdf', 'Unknown_EDS_2.pdf', 'Unknown_EDS_3.pdf', 'Unknown_EDS_4.pdf', 'Unknown_EDS_5.pdf']


In [17]:
from utils import get_doc_tools
from pathlib import Path

# Ensure function names are within the allowed length limit
def truncate_function_name(name, max_length=64):
    return name if len(name) <= max_length else name[:max_length]

paper_to_tools_dict = {}
for pdf in pdf_files:
    print(f"Getting tools for paper: {pdf}")
    vector_tool, summary_tool = get_doc_tools(pdf, Path(pdf).stem)
    #vector_tool, summary_tool = get_doc_tools(pdf, truncate_function_name(Path(pdf).stem))
    paper_to_tools_dict[pdf] = [vector_tool, summary_tool]
    #print(vector_tool)
    #print(summary_tool)

Getting tools for paper: Ehlers-Danlos-1\2024_EDS_1.pdf
Getting tools for paper: Ehlers-Danlos-1\2024_EDS_2.pdf
Getting tools for paper: Ehlers-Danlos-1\2024_EDS_3.pdf
Getting tools for paper: Ehlers-Danlos-1\2024_EDS_4.pdf
Getting tools for paper: Ehlers-Danlos-1\2024_EDS_5.pdf
Getting tools for paper: Ehlers-Danlos-1\2024_EDS_6.pdf
Getting tools for paper: Ehlers-Danlos-1\2024_EDS_7.pdf
Getting tools for paper: Ehlers-Danlos-1\Unknown_EDS_1.pdf
Getting tools for paper: Ehlers-Danlos-1\Unknown_EDS_2.pdf
Getting tools for paper: Ehlers-Danlos-1\Unknown_EDS_3.pdf
Getting tools for paper: Ehlers-Danlos-1\Unknown_EDS_4.pdf
Getting tools for paper: Ehlers-Danlos-1\Unknown_EDS_5.pdf


In [18]:
all_tools = [t for pdf in pdf_files for t in paper_to_tools_dict[pdf]]
#all_tools = [truncate_function_name(tool) for tool in all_tools]


In [19]:
# define an "object" index and retriever over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [20]:
obj_retriever = obj_index.as_retriever(similarity_top_k=3)


In [21]:
tools = obj_retriever.retrieve(
    "How do I tell if someone has EDS?"
)

In [22]:
print(len(tools))
tools[2].metadata


3


ToolMetadata(description='Useful for summarization questions related to Unknown_EDS_3', name='summary_tool_Unknown_EDS_3', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)

In [23]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")

In [24]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_retriever,
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [25]:

response = agent.query(
    "How do I tell if someone has EDS?"
)
print(str(response))

Added user message to memory: How do I tell if someone has EDS?
=== LLM Response ===
To determine if someone has Ehlers-Danlos syndrome (EDS), a healthcare professional typically conducts a thorough medical evaluation. The diagnosis of EDS involves a combination of clinical assessment, family history review, physical examination, and sometimes genetic testing. Here are some common steps involved in diagnosing EDS:

1. **Medical History**: The healthcare provider will ask about the individual's symptoms, medical history, and family history of EDS or related conditions.

2. **Physical Examination**: A comprehensive physical examination will be conducted to assess joint flexibility, skin elasticity, and other physical features associated with EDS.

3. **Beighton Score**: The Beighton score is often used to evaluate joint hypermobility. It involves assessing the flexibility of specific joints through a series of movements.

4. **Genetic Testing**: In some cases, genetic testing may be reco